# RESEARCH NOTEBOOK --> SmugPlug

In [ ]:
# This is necessary to recognize the modules
import os
import sys
from decimal import Decimal
import warnings

warnings.filterwarnings("ignore")

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

In [ ]:
import pandas as pd
import pandas_ta as ta  # noqa: F401
from core.data_sources import CLOBDataSource

# Initialize the data source
clob = CLOBDataSource()

In [ ]:
# Define the parameters
exchange = "binance_perpetual"
trading_pair = "WLD-USDT"
timeframe = "1h"
days = 30

In [ ]:
# Get the candles
candles = await clob.get_candles_last_days(
    exchange, trading_pair, timeframe, days, from_trades=False)

In [ ]:
candles.data

In [ ]:
candles.plot(type="returns", height=400, width=800)

In [ ]:
# EMAs
ema_short = 8
ema_medium = 29
ema_long = 31

# MACD
macd_fast = 22
macd_slow = 36
macd_signal = 17

# ATR
atr_length = 11
atr_multiplier = 1.5

# Add indicators
candles_df = candles.data
candles_df.ta.macd(fast=macd_fast, slow=macd_slow, signal=macd_signal, append=True)
candles_df.ta.atr(length=atr_length, append=True)
candles_df.ta.ema(length=ema_short, append=True)
candles_df.ta.ema(length=ema_medium, append=True)
candles_df.ta.ema(length=ema_long, append=True)
candles_df["long_atr_support"] = candles_df["close"].shift(1) - candles_df[f"ATRr_{atr_length}"] * atr_multiplier
candles_df["short_atr_resistance"] = candles_df["close"].shift(1) + candles_df[f"ATRr_{atr_length}"] * atr_multiplier

candles_df.tail(5)

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.03, 
                    subplot_titles=(trading_pair, 'MACD'),
                    row_heights=[0.7, 0.3])

# Add candlestick
fig.add_trace(go.Candlestick(x=candles_df.index,
                             open=candles_df['open'],
                             high=candles_df['high'],
                             low=candles_df['low'],
                             close=candles_df['close'],
                             name='OHLC'),
              row=1, col=1)

# Add EMAs
ema_fast = f'EMA_{ema_short}'
ema_med = f'EMA_{ema_medium}' 
ema_slow = f'EMA_{ema_long}'

fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df[ema_fast],
                         line=dict(color='#00FF00', width=2),
                         name='Fast EMA'), row=1, col=1)
fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df[ema_med],
                         line=dict(color='#FFA500', width=2), 
                         name='Medium EMA'), row=1, col=1)
fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df[ema_slow],
                         line=dict(color='#0000FF', width=2),
                         name='Slow EMA'), row=1, col=1)

# Add MACD
macd = f'MACD_{macd_fast}_{macd_slow}_{macd_signal}'
macd_s = f'MACDs_{macd_fast}_{macd_slow}_{macd_signal}'
macd_hist = f'MACDh_{macd_fast}_{macd_slow}_{macd_signal}'

fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df[macd], 
                         line=dict(color='#00FFFF', width=2),
                         name='MACD'), row=2, col=1)
fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df[macd_s], 
                         line=dict(color='#FFA500', width=2),
                         name='Signal'), row=2, col=1)
fig.add_trace(go.Bar(x=candles_df.index, y=candles_df[macd_hist], name='Histogram',
                     marker_color=candles_df[macd_hist].apply(
                         lambda x: '#00FF00' if x >= 0 else '#FF0000')),
                    row=2, col=1)

# Add ATR support and resistance
fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df["long_atr_support"],
                         line=dict(color='#00FF00', width=2),
                         name='Long ATR Support'), row=1, col=1)
fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df["short_atr_resistance"],
                         line=dict(color='#FF0000', width=2),
                         name='Short ATR Resistance'), row=1, col=1)

# Update layout for dark theme
fig.update_layout(
    title=f'{exchange} - {trading_pair} - {timeframe}',
    width=1200, height=800,
    font=dict(color='#e1e1e1'),
    plot_bgcolor='#1e1e1e',
    paper_bgcolor='#1e1e1e',
    xaxis_rangeslider_visible=False,
    legend=dict(bgcolor='rgba(0,0,0,0)'),
    yaxis=dict(title='Price'),
    yaxis2=dict(title='MACD', showgrid=False),
    showlegend=False
)

# Update axes
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='#323232', zeroline=False)
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#323232', zeroline=False)

# Show the plot
fig.show()

In [ ]:
# Generate signal


macdh = candles_df[f"MACDh_{macd_fast}_{macd_slow}_{macd_signal}"]
short_ema = candles_df[f"EMA_{ema_short}"]
medium_ema = candles_df[f"EMA_{ema_medium}"]
long_ema = candles_df[f"EMA_{ema_long}"]
close = candles_df["close"]


long_condition = (short_ema > medium_ema) & (medium_ema > long_ema) & (close > short_ema) & (close > candles_df["long_atr_support"]) & (macdh > 0) 
short_condition = (short_ema < medium_ema) & (medium_ema < long_ema) & (close < short_ema) & (close < candles_df["short_atr_resistance"]) & (macdh < 0)

candles_df["signal"] = 0
candles_df.loc[long_condition, "signal"] = 1
candles_df.loc[short_condition, "signal"] = -1

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.02,
                    subplot_titles=('OHLC with BB', 'MACD', 'Signal'),
                    row_heights=[0.6, 0.2, 0.2])

# Add candlestick
fig.add_trace(go.Candlestick(x=candles_df.index,
                             open=candles_df['open'],
                             high=candles_df['high'],
                             low=candles_df['low'],
                             close=candles_df['close'],
                             name='Candlesticks'),
              row=1, col=1)


# Add EMAs
ema_fast = f'EMA_{ema_short}'
ema_med = f'EMA_{ema_medium}' 
ema_slow = f'EMA_{ema_long}'

fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df[ema_fast],
                         line=dict(color='#00FF00', width=2),
                         name='Fast EMA'), row=1, col=1)
fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df[ema_med],
                         line=dict(color='#FFA500', width=2), 
                         name='Medium EMA'), row=1, col=1)
fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df[ema_slow],
                         line=dict(color='#0000FF', width=2),
                         name='Slow EMA'), row=1, col=1)

# Add MACD
macd = f'MACD_{macd_fast}_{macd_slow}_{macd_signal}'
macd_s = f'MACDs_{macd_fast}_{macd_slow}_{macd_signal}'
macd_hist = f'MACDh_{macd_fast}_{macd_slow}_{macd_signal}'

fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df[macd], 
                         line=dict(color='#00FFFF', width=2),
                         name='MACD'), row=2, col=1)
fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df[macd_s], 
                         line=dict(color='#FFA500', width=2),
                         name='Signal'), row=2, col=1)
fig.add_trace(go.Bar(x=candles_df.index, y=candles_df[macd_hist], name='Histogram',
                     marker_color=candles_df[macd_hist].apply(
                         lambda x: '#00FF00' if x >= 0 else '#FF0000')),
              row=2, col=1)

# Add ATR support and resistance
fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df["long_atr_support"],
                         line=dict(color='#00FF00', width=2),
                         name='Long ATR Support'), row=1, col=1)
fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df["short_atr_resistance"],
                         line=dict(color='#FF0000', width=2),
                         name='Short ATR Resistance'), row=1, col=1)

# Add the signal line
fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df['signal'],
                         mode='lines',
                         name='Signal',
                         line=dict(color="white")),
              row=3, col=1)

# Update layout for dark theme
fig.update_layout(
    title=f'{exchange} - {trading_pair} - {timeframe}',
    width=1500, height=1000,
    font=dict(color='#e1e1e1'),
    plot_bgcolor='#1e1e1e',
    paper_bgcolor='#1e1e1e',
    xaxis_rangeslider_visible=False,
    legend=dict(bgcolor='rgba(0,0,0,0)'),
    yaxis=dict(title='Price'),
    yaxis2=dict(title='MACD', showgrid=False),
    yaxis3=dict(title='Signal', showgrid=False),
    showlegend=False
)

# Update axes
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='#323232', zeroline=False)
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#323232', zeroline=False)

# Show the plot
fig.show()


# CONCLUSION

In this notebook, we have implemented a strategy combining the MACD (Moving Average Convergence Divergence) indicator with Bollinger Bands. We've visualized these indicators along with the price data and generated signals based on their interactions. This approach provides a solid foundation for our trading strategy.
 
## Key components of our strategy include:
 1. MACD for trend identification
 2. Bollinger Bands for volatility measurement and potential reversal points
 3. A signal line derived from the combination of these indicators
 
 The next step is to backtest this strategy to evaluate its profitability and robustness. For this purpose, we have created a controller file named `macd_bb.py` in this folder. This file implements the logic we've developed here, allowing us to conduct comprehensive backtests in the subsequent notebook.